# Tunier


In [1]:
import os.path
css = ""
if os.path.isfile("style.html"):
    from IPython.core.display import HTML
    with open("style.html", "r") as file:
        css = file.read()
HTML(css)

In [2]:
%run ./nmm-game.ipynb
%run ./nmm-artificial-intelligence.ipynb
%run ./nmm-minimax.ipynb
%run ./nmm-alpha-beta-pruning.ipynb

## Runde


In [3]:
class Round():
    def __init__(self, white: ArtificialIntelligence, black: ArtificialIntelligence, start_state = s0, start_player = 'w', max_turns: int = 250, name: str = ""):
        self.white = white
        self.black = black
        
        self.state = start_state
        self.player = start_player
        self.max_turns = max_turns
        self.name = name
        
        self.log = []

In [4]:
def __repr__(self: Round):
    return f"Round(name='{self.name}', white={type(self.white).__name__}, black={type(self.black).__name__}, max_turns={self.max_turns})"

Round.__repr__ = __repr__
del __repr__

In [5]:
class RoundResult():
    def __init__(self, winner: str, turns: int, reason: str):
        self.winner = winner
        self.turns  = turns
        self.reason = reason
    
    def __repr__(self):
        return f"RoundResult(winner='{self.winner}', turns={self.turns}, reason='{self.reason}')"

In [6]:
def current_ai(self: Round) -> ArtificialIntelligence:
    if self.player == 'w':
        return self.white
    return self.black

Round.current_ai = current_ai
del current_ai

In [7]:
def play(self: Round) -> RoundResult:
    for turn in range(self.max_turns):
        if finished(self.state, self.player):
            return RoundResult(
                winner = ('w', ' ', 'b')[utility(self.state, 'b')+1],
                turns  = turn+1,
                reason = f"finished() returned true"
            )
        
        bestMoves = self.current_ai().bestMoves(self.state, self.player)
        self.log.append(bestMoves)
        
        self.state  = bestMoves.choice()
        self.player = opponent(self.player)
        
        print(f"{self.name} t={turn+1: >3} {self.player} found {len(bestMoves.states): >2} states with value={bestMoves.value}")
    
    return RoundResult(
        winner=' ',
        turns=self.max_turns,
        reason=f"Reached max_turns after {self.max_turns} turns"
    )

Round.play = play
del play

## Wettkampf


In [8]:
from multiprocessing import cpu_count, Pool

In [9]:
class Tournament():
    def __init__(self, white: ArtificialIntelligence, black: ArtificialIntelligence, instances: int = None):
        self.white = white
        self.black = black
        
        self.instances = instances
        if self.instances is None:
            self.instances = cpu_count()
        
        self.rounds = [
            Round(
                self.white() if callable(self.white) else self.white,
                self.black() if callable(self.black) else self.black,
                name = f"r={i: <2}"
            )
            for i in range(self.instances)
        ]

In [10]:
def execute(args):
    index, cround = args
    random.seed(index)
    print("hmm")
    return cround.play()

In [11]:
def play(self: Tournament):
    print("hööö")
    with Pool(len(self.rounds)) as pool:
        print("hii")
        return pool.map(execute, enumerate(self.rounds))

Tournament.play = play
del play

In [12]:
t = Tournament(Minimax(limit=1), Minimax(limit=1))

In [13]:
t.rounds

[Round(name='r=0 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=1 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=2 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=3 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=4 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=5 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=6 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=7 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=8 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=9 ', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=10', white=Minimax, black=Minimax, max_turns=250),
 Round(name='r=11', white=Minimax, black=Minimax, max_turns=250)]

In [14]:
%%time
t.play()

hööö
hii


PicklingError: Can't pickle <class '__main__.HeuristicWeights'>: it's not the same object as __main__.HeuristicWeights